Make a composite image

In [1]:
import tkinter as tk
from tkinter import filedialog
import json
import numpy as np
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

file_path = filedialog.askopenfilename(title = "Choose the position list", initialdir = "D:/Experiments/MUSCLE DONUTS/20220824_FC_Nano_Cas9_CCR5/parameter")
pos_file = open(file_path)
data = json.load(pos_file)
POS = data['POSITIONS']

labels = [P['LABEL'] for P in POS]
posX = [P['DEVICES'][0]['X'] for P in POS]
posY = [P['DEVICES'][0]['Y'] for P in POS]

from skimage import transform
tr = transform.SimilarityTransform()
tr.params = np.array([[ 5.98278480e-01, -2.04811207e-03,  0],
 [ 2.04811207e-03,  5.98278480e-01,  0],
 [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])
tr_inv = transform.SimilarityTransform()
tr_inv.params = np.array([[ 1.67127284e+00,  5.72133913e-03,  0],
 [-5.72133913e-03,  1.67127284e+00,  0],
 [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

import os.path as path
import skimage as si
import skimage.io as io


size1 = tr([[256,512]]).astype(int)
print(size1)

minx = min(posX)
maxx = max(posX)
miny = min(posY)
maxy = max(posY)

size = [maxy-miny+50+size1[0,0], maxx-minx+50+size1[0,1]]
size = np.divide(size,0.34).astype(int)
print(size)

img = np.zeros(size)

rb_rad = 10         

red_path = filedialog.askdirectory(title = "Choose the bead directory", initialdir = "D:/Experiments/MUSCLE DONUTS/20220824_FC_Nano_Cas9_CCR5/parameter")

for pos in labels:
    path1 = path.join(red_path,pos,'A_Red','img_000000000.tiff')
#     path1 = path.join(red_path,pos+'.tiff')
    if path.exists(path1):
        print(pos)
        img1 = io.imread(path1)
        if img1.shape[0] == 512: # sometimes slice number is the first dimension, sometimes the third
            img1 = np.mean(img1, axis = 2) #averaging by the stack
        else:
            img1 = np.mean(img1, axis = 0) #averaging by the stack
#         img1 = np.mean(img1, axis = 0)
#         img1 = np.mean(img1, axis = 2)
        img1 = img1.astype("ushort")
        img1 = img1[256:,:]
        img1 = img1 - si.restoration.rolling_ball(img1, radius=rb_rad)
        img1 = transform.warp(img1, tr_inv, output_shape = size1[0])
        x1 = [posX[i] for i,x in enumerate(labels) if labels[i]==pos]
        y1 = [posY[i] for i,x in enumerate(labels) if labels[i]==pos]
        id1 = int((maxy-y1[0])/0.34)
        id2 = int((x1[0]-minx)/0.34)

        img[id1:id1+size1[0,0],id2:id2+size1[0,1]] = img1
        
        
from skimage.util import img_as_uint, img_as_ubyte
img2 = img_as_uint(img)
print(img2)
io.imsave(red_path+'/Combined.png',img2)

print("DONE")

# END OF THE MAIN CYCLE

In [78]:
pos = 'Pos120'
x1 = [posX[i] for i,x in enumerate(labels) if labels[i]==pos]
y1 = [posY[i] for i,x in enumerate(labels) if labels[i]==pos]
id1 = int((maxy-y1[0])/0.34)
id2 = int((x1[0]-minx)/0.34)
print(x1,y1,id1,id2,labels[0])

[27619.100000000002] [-1588.2] 2647 4205 Pos103


In [18]:
# x0 = 28523
# y0 = -1313
# idx = []
# for i,lab in enumerate(labels):
#     if abs(posX[i]-x0)<200 and abs(posY[i]-y0)<200:
#         idx.append(lab)


In [19]:

# x1 = [posX[i] for i,x in enumerate(labels) if labels[i]=='Pos246']
# y1 = [posY[i] for i,x in enumerate(labels) if labels[i]=='Pos246']
# print(x1, y1)
# # x1_old = [posX[i] for i,x in enumerate(labels) if labels[i]=='Pos246']
# # y1_old = [posY[i] for i,x in enumerate(labels) if labels[i]=='Pos246']
    
# # print(2329+(x1[0]-28523)/0.34)
# # print(2042-(y1[0]+1313)/0.34)

In [5]:
# pos = labels[10]
# path1 = path.join(red_path,pos,'A_Red','img_000000000.tiff')
# img1 = io.imread(path1)
# print(img1.shape)

(512, 512, 3)


In [11]:
# path1 = path.join(red_path,pos,'A_Red','img_000000000.tiff')
# img1 = io.imread(path1)   
# print(img1.shape)
# pos = "Pos400"
# path1 = path.join(red_path,pos,'A_Red','img_000000000.tiff')
# img1 = io.imread(path1)

(512, 512, 3)


In [8]:
# img2 = np.mean(img1, axis = 2)
        
# print(img2.shape)